In [42]:
from glob import glob

import pandas as pd

#최종 df 먼저 생성 
df = []
datas = glob(f'./datas/2024/*.xlsx')

teams = [xlsx.split('_')[-2] for xlsx in datas]

# concat 기준이 될 df 먼저 생성, 
df_year = pd.read_excel(f'{datas[0]}', engine='openpyxl')

# H/A 헷갈려서 기준 잡는 열 생성 / 시즌 열 생성
df_year['기준'] = teams[0]
df_year['시즌'] = 2024

# 새로운 df 불러오고 기준 df에 concat하기
for xlsx in datas[1:]:
    team_name = xlsx.split('_')[-2]
    df_temp = pd.read_excel(f'{xlsx}', engine="openpyxl")
    df_temp["기준"] = f'{team_name}'
    df_temp["시즌"] = 2024
    df_year = pd.concat([df_year, df_temp])
df.append(df_year)

df = pd.concat(df, ignore_index=True)


# K_Columns 에서 컬럼명 가져오기
names = pd.read_excel('datas/K_columns.xlsx', engine="openpyxl")


# 필요없는 열 제거 (성공%만 가져옴)
temp = names.drop(columns=['LABEL-45','LABEL-46','LABEL-47','LABEL-48']) 


# 컬럼명 가져와서 리스트로 저장
rows = temp.iloc[0].to_list()


# df의 컬럼 개수를 K_columns에 맞춰 개수 맞추기
drop_columns = ['14', '15','20','21','23','24','27', '28' ,'30' ,'31' ,'33' ,'34', '36', '37', '39', '40','42', '43' ,'45',
               '46' ,'48', '49' ,'51' ,'52' ,'54', '55', '58', '59', '61', '62', '64', '65']
for idx in (drop_columns):
    df = df.drop(columns=[f'LABEL-{idx}'])


# 컬럼명 변경 ("기준"열 제외한)
for idx, column in enumerate(df.columns[:-2]):
    df = df.rename(columns={f"{column}": f"{rows[idx]}"})


# 대회별 득점 비교 후 경기결과 열에 저장
for i in df["대회"].value_counts().index.tolist():
    for j in range(1,39): # 총 35개 라운드 수집
        H_score = df[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME")]["득점"]
        A_score = df[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY")]["득점"]
        if len(H_score) > 0:
            if H_score.iloc[0] > A_score.iloc[0]:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'W'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'L'
            elif H_score.iloc[0] < A_score.iloc[0]:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'L'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'W'
            else:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'D'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'D'


# 컬럼명 변경 확인
df.head()

,Rnd.,대회,H/A,득점,도움,슈팅,유효 슈팅,블락된슈팅,벗어난슈팅,PA내 슈팅,...,획득,블락,볼미스,파울,피파울,경고,퇴장,기준,시즌,경기결과
0,1,대구vs김천,HOME,0,0,16,3,4,9,9,...,99,22,8,11,8,1,0,daegu,2024,L
1,2,포항vs대구,AWAY,1,0,12,2,3,7,4,...,72,15,4,10,21,1,0,daegu,2024,L
2,3,대구vs수원FC,HOME,1,0,13,8,1,4,9,...,63,21,3,7,4,1,0,daegu,2024,D
3,4,광주vs대구,AWAY,2,1,13,4,3,6,6,...,81,29,4,8,13,0,0,daegu,2024,W
4,5,강원vs대구,AWAY,0,0,9,0,5,4,3,...,80,12,3,17,16,2,0,daegu,2024,L


In [43]:
tm_data_2024 = pd.read_excel('datas/K_trsfmk_2024.xlsx', engine='openpyxl').drop(columns=['LABEL-1','LABEL-2','LABEL-4'])

tm_data_2024 = tm_data_2024.rename(columns={'LABEL-3' : '구단', 'LABEL-5' : '소속 선수 수',
                                'LABEL-6' : '평균 나이','LABEL-7' : '용병 수',
                                'LABEL-8' : '평균 가치', 'LABEL-9' : '총 가치'})

tm_data_2024['구단'] = ['gimcheon', 'jeonbuk', 'seoul', 'ulsan', 'daejeon', 'gwangju',
                           'incheon', 'gangwon', 'jeju', 'suwonFC','daegu','pohang']

tm_data_2024['평균 가치'] = tm_data_2024['평균 가치'].str.replace('€', '').str.replace('k', '').astype(float)
tm_data_2024['총 가치'] = tm_data_2024['총 가치'].str.replace('€', '').str.replace('m', '').astype(float)



In [44]:
tm_columns = tm_data_2024.columns[1:]

In [47]:
# df에 열 먼저 생성해주기
for column in tm_columns:
    df[f'{column}'] = None

#2024
for column in tm_data_2024.columns[1:]:
    for i in df['기준']:
        for j in tm_data_2024['구단']:
            if i == j:
                df.loc[(df['기준'] == i) &(df['시즌'] == 2024), f'{column}'] = tm_data_2024.loc[tm_data_2024['구단'] == j , f'{column}'].values[0]
print('2024 end !')
df.tail()

2024 end !


,Rnd.,대회,H/A,득점,도움,슈팅,유효 슈팅,블락된슈팅,벗어난슈팅,PA내 슈팅,...,경고,퇴장,기준,시즌,경기결과,소속 선수 수,평균 나이,용병 수,평균 가치,총 가치
175,11,서울vs울산,AWAY,1,0,9,3,1,5,9,...,2,0,ulsan,2024,W,34,28.6,6,502.0,17.08
176,12,울산vs김천,HOME,2,2,11,6,1,4,7,...,1,0,ulsan,2024,D,34,28.6,6,502.0,17.08
177,13,강원vs울산,AWAY,0,0,12,4,2,6,9,...,1,0,ulsan,2024,L,34,28.6,6,502.0,17.08
178,14,울산vs대전,HOME,4,1,30,13,3,14,24,...,1,0,ulsan,2024,W,34,28.6,6,502.0,17.08
179,15,인천vs울산,AWAY,1,1,5,3,1,1,4,...,5,0,ulsan,2024,D,34,28.6,6,502.0,17.08


In [48]:
new_rows = []

unique_competitions = df['대회'].unique()

for competition in unique_competitions:
    competition_rows = df[df['대회'] == competition]
    
    unique_seasons = competition_rows['시즌'].unique()
    
    for season in unique_seasons:
        season_rows = competition_rows[competition_rows['시즌'] == season]
        
        unique_rounds = season_rows['Rnd.'].unique()
        
        for rnd in unique_rounds:
            round_rows = season_rows[season_rows['Rnd.'] == rnd]
            
            home_row = round_rows[round_rows['H/A'] == 'HOME'].iloc[0]
            away_row = round_rows[round_rows['H/A'] == 'AWAY'].iloc[0]
            
            merged_row = pd.concat([home_row, away_row]).reset_index(drop=True)
            new_rows.append(merged_row)

new_df = pd.DataFrame(new_rows)

# new_df.head()

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
0,1,대구vs김천,HOME,0,0,16,3,4,9,9,...,3,0,gimcheon,2024,W,53,25.1,0,435.0,23.05
1,2,포항vs대구,HOME,3,2,9,6,2,1,6,...,1,0,daegu,2024,L,43,23.8,5,210.0,9.05
2,3,대구vs수원FC,HOME,1,0,13,8,1,4,9,...,1,0,suwonFC,2024,D,43,24.7,4,224.0,9.63
3,4,광주vs대구,HOME,1,0,10,4,1,5,9,...,0,0,daegu,2024,W,43,23.8,5,210.0,9.05
4,5,강원vs대구,HOME,3,1,12,6,1,5,4,...,2,0,daegu,2024,L,43,23.8,5,210.0,9.05


In [50]:
# 컬럼 명 합치기

merged_columns = ['Rnd.', '대회', 'H/A', '홈 득점', '홈 도움', '홈 슈팅', '홈 유효 슈팅', '홈 블락된슈팅', '홈 벗어난슈팅',
       '홈 PA내 슈팅', '홈 PA외 슈팅', '홈 프리킥 슈팅', '홈 프리킥 유효슈팅', '홈 프리킥 크로스', '홈 오프사이드', '홈 코너킥',
       '홈 스로인', '홈 드리블', '홈 패스', '홈 키패스', '홈 전방패스', '홈 후방패스', '홈 횡패스', '홈 공격진영 패스', '홈 수비진영 패스',
       '홈 중앙진영 패스', '홈 롱패스', '홈 중거리패스', '홈 단거리패스', '홈 크로스', '홈 탈압박', '홈 경합 지상', '홈 경합 공중',
       '홈 태클', '홈 클리어링', '홈 인터셉트', '홈 차단', '홈 획득', '홈 블락', '홈 볼미스', '홈 파울', '홈 피파울', '홈 경고', '홈 퇴장',
       '기준', '시즌', '홈 경기결과', '홈 소속 선수 수', '홈 평균 나이', '홈 용병 수', '홈 평균 가치', '홈 총 가치',
       
       'Rnd.2', '대회2', 'H/A2', '원정 득점', '원정 도움', '원정 슈팅', '원정 유효 슈팅', '원정 블락된슈팅', '원정 벗어난슈팅',
       '원정 PA내 슈팅', '원정 PA외 슈팅', '원정 프리킥 슈팅', '원정 프리킥 유효슈팅', '원정 프리킥 크로스', '원정 오프사이드', '원정 코너킥',
       '원정 스로인', '원정 드리블', '원정 패스', '원정 키패스', '원정 전방패스', '원정 후방패스', '원정 횡패스', '원정 공격진영 패스', '원정 수비진영 패스',
       '원정 중앙진영 패스', '원정 롱패스', '원정 중거리패스', '원정 단거리패스', '원정 크로스', '원정 탈압박', '원정 경합 지상', '원정 경합 공중',
       '원정 태클', '원정 클리어링', '원정 인터셉트', '원정 차단', '원정 획득', '원정 블락', '원정 볼미스', '원정 파울', '원정 피파울', '원정 경고', '원정 퇴장',
       '기준2', '시즌2', '원정 경기결과', '원정 소속 선수 수', '원정 평균 나이', '원정 용병 수', '원정 평균 가치', '원정 총 가치']


In [51]:
new_df.columns = merged_columns

# 불필요 컬럼 제거
new_df = new_df.drop(columns=['Rnd.2', '대회2', '시즌2', 'H/A', 'H/A2'])
new_df = new_df.rename(columns={'기준':'홈 팀', '기준2':'원정 팀'})
new_df

,Rnd.,대회,홈 득점,홈 도움,홈 슈팅,홈 유효 슈팅,홈 블락된슈팅,홈 벗어난슈팅,홈 PA내 슈팅,홈 PA외 슈팅,...,원정 피파울,원정 경고,원정 퇴장,원정 팀,원정 경기결과,원정 소속 선수 수,원정 평균 나이,원정 용병 수,원정 평균 가치,원정 총 가치
0,1,대구vs김천,0,0,16,3,4,9,9,7,...,11,3,0,gimcheon,W,53,25.1,0,435.0,23.05
1,2,포항vs대구,3,2,9,6,2,1,6,3,...,21,1,0,daegu,L,43,23.8,5,210.0,9.05
2,3,대구vs수원FC,1,0,13,8,1,4,9,4,...,7,1,0,suwonFC,D,43,24.7,4,224.0,9.63
3,4,광주vs대구,1,0,10,4,1,5,9,1,...,13,0,0,daegu,W,43,23.8,5,210.0,9.05
4,5,강원vs대구,3,1,12,6,1,5,4,8,...,16,2,0,daegu,L,43,23.8,5,210.0,9.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,13,수원FCvs포항,1,1,10,4,2,4,5,5,...,6,2,0,pohang,L,40,24.8,4,220.0,8.80
86,14,포항vs서울,2,0,6,3,1,2,2,4,...,9,3,0,seoul,D,41,24.9,6,430.0,17.63
87,10,수원FCvs서울,0,0,8,5,1,2,7,1,...,6,1,0,seoul,W,41,24.9,6,430.0,17.63
88,11,서울vs울산,0,0,10,6,2,2,7,3,...,10,2,0,ulsan,W,34,28.6,6,502.0,17.08


In [52]:
new_df.to_csv('K-League-TEST-data.csv')